In [ ]:
import pandas as pd
from sklearn.metrics import f1_score
#1000条数据试验词向量转换功能，转换为FastText需要的格式
train_df = pd.read_csv('./train_set.csv', sep='\t', nrows=1000)
train_df['label_ft'] = '__label__' + train_df['label'].astype(str)
train_df[['text','label_ft']].iloc[:-500].to_csv('train1000.csv', index=None, header=None, sep='\t')

import fasttext
model1 = fasttext.train_unsupervised('train1000.csv', lr=0.1, wordNgrams=2, 
                                  verbose=2, minCount=1, epoch=2, loss="hs")


In [2]:
model1.save_model("result1000.bin")
#cmd命令行执行python bin_to_vec.py result1000.bin < result1000.vec，转换为vec词向量

model2 = fasttext.train_supervised('train1000.csv',pretrainedVectors='result1000.vec',lr=1.0, wordNgrams=2, 
                                  verbose=2, minCount=1, epoch=2, loss="hs")
#试验成功

正式测试开始，选取15000条数据。前1w条训练，后5000条测试

In [14]:
#正常fasttext训练
import pandas as pd
from sklearn.metrics import f1_score

# 转换为FastText需要的格式
train_df = pd.read_csv('./train_set.csv', sep='\t', nrows=15000)
train_df['label_ft'] = '__label__' + train_df['label'].astype(str)
train_df[['text','label_ft']].iloc[:-5000].to_csv('train10000.csv', index=None, header=None, sep='\t')

import fasttext
model = fasttext.train_supervised('train.csv', lr=1.0, wordNgrams=2, 
                                  verbose=2, minCount=1, epoch=25, loss="hs")

val_pred = [model.predict(x)[0][0].split('__')[-1] for x in train_df.iloc[-5000:]['text']]
print(f1_score(train_df['label'].values[-5000:].astype(str), val_pred, average='macro'))

0.8271753451328839


In [17]:
#先进行word2vec训练，使用全部15000数据
train_df[['text','label_ft']].to_csv('train15000.csv', index=None, header=None, sep='\t')
model1 = fasttext.train_unsupervised('train15000.csv', lr=0.1, wordNgrams=2, 
                                  verbose=2, minCount=1, epoch=8, loss="hs")
#保存模型转为词向量
model1.save_model("word15000.bin")
#cmd命令行执行python bin_to_vec.py word15000.bin < word15000.vec，转换为vec词向量

In [4]:
#fasttext进行1w条数据训练
"""import fasttext
import pandas as pd
from sklearn.metrics import f1_score
model2 = fasttext.train_supervised('train.csv',pretrainedVectors='word15000.vec',lr=1.0, wordNgrams=2, 
                                 verbose=2, minCount=1, epoch=16, loss="hs")"""
#预测结果
train_df = pd.read_csv('./train_set.csv', sep='\t', nrows=15000)
val_pred = [model2.predict(x)[0][0].split('__')[-1] for x in train_df.iloc[-5000:]['text']]
print(f1_score(train_df['label'].values[-5000:].astype(str), val_pred, average='macro'))

0.8502375081398378


In [ ]:
#首尾截断实验效果
#准备将text文本首尾截断，各取255tokens
def slipt2(x):
  ls=x.split(' ')
  le=len(ls)
  if le<201:
    return x
  else:
    return ' '.join(ls[:100]+ls[-100:])


In [28]:
#首尾截断进行训练
train_df['summary']=train_df['text'].apply(lambda x:slipt2(x))
train_df[['summary','label_ft']].iloc[:-5000].to_csv('train_summary10000.csv', index=None, header=None, sep='\t')

model3 = fasttext.train_supervised('train_summary10000.csv',pretrainedVectors='word15000.vec',lr=1.0, wordNgrams=2, 
                                  verbose=2, minCount=1, epoch=16, loss="hs")
#预测结果
val_pred = [model3.predict(x)[0][0].split('__')[-1] for x in train_df.iloc[-5000:]['summary']]
print(f1_score(train_df['label'].values[-5000:].astype(str), val_pred, average='macro'))

0.8304409751791343


In [6]:
model2.save_model("fastword15000.bin")
#cmd命令行执行python bin_to_vec.py fastword15000.bin < fastword15000.vec，转换为vec词向量
#对比fasttext_word15000.vec和word15000.vec词向量，两者都是（5516 100），但是词向量已经不一样了。

------------------------------------------分割线啊---------------------------------

开始正式训练


In [ ]:
#读取训练测试集数据
import pandas as pd
from sklearn.metrics import f1_score

# 转换为FastText需要的格式
train_df = pd.read_csv('./train_set.csv', sep='\t')
train_df['label_ft'] = '__label__' + train_df['label'].astype(str)
train_df[['text','label_ft']].to_csv('train_20w.csv', index=None, header=None, sep='\t')

test_df = pd.read_csv('./test_a.csv', sep='\t')
df=pd.concat([train_df,test_df])
df[['text']].to_csv('train_25w.csv', index=None, header=None, sep='\t')

In [1]:
import fasttext
#word2vec进行train+test词向量训练
model1 = fasttext.train_unsupervised('train_25w.csv', lr=0.1, wordNgrams=2, 
                                  verbose=2, minCount=1, epoch=8, loss="hs")

model1.save_model("word_25w.bin")
#python bin_to_vec.py word_25w.bin > word_25w.vec

In [4]:
#fasttext进行有监督训练
#model2 = fasttext.train_supervised('train_20w.csv',pretrainedVectors='word_25w.vec',lr=0.8, wordNgrams=2, 
#                                 verbose=2, minCount=1, epoch=18, loss="hs")
import pandas as pd
test_df = pd.read_csv('./test_a.csv', sep='\t')
test_pred = [model2.predict(x)[0][0].split('__')[-1] for x in test_df['text']]

pd.DataFrame({'label':test_pred}).to_csv('word_fast.csv',index=None)

In [59]:
#首尾截断实验效果
#准备将text文本首尾截断，各取100tokens
def slipt2(x):
    ls=x.split(' ')
    le=len(ls)
    if le<301:
        return x
    else:
        return ' '.join(ls[:150]+ls[-150:])

In [56]:
train_df

,label,text,label_ft,summary
0,2,2967 6758 339 2021 1854 3731 4109 3792 4149 15...,__label__2,2967 6758 339 2021 1854 3731 4109 3792 4149 15...
1,11,4464 486 6352 5619 2465 4802 1452 3137 5778 54...,__label__11,4464 486 6352 5619 2465 4802 1452 3137 5778 54...
2,3,7346 4068 5074 3747 5681 6093 1777 2226 7354 6...,__label__3,7346 4068 5074 3747 5681 6093 1777 2226 7354 6...
3,2,7159 948 4866 2109 5520 2490 211 3956 5520 549...,__label__2,7159 948 4866 2109 5520 2490 211 3956 5520 549...
4,3,3646 3055 3055 2490 4659 6065 3370 5814 2465 5...,__label__3,3646 3055 3055 2490 4659 6065 3370 5814 2465 5...
...,...,...,...,...
199995,2,307 4894 7539 4853 5330 648 6038 4409 3764 603...,__label__2,307 4894 7539 4853 5330 648 6038 4409 3764 603...
199996,2,3792 2983 355 1070 4464 5050 6298 3782 3130 68...,__label__2,3792 2983 355 1070 4464 5050 6298 3782 3130 68...
199997,11,6811 1580 7539 1252 1899 5139 1386 3870 4124 1...,__label__11,6811 1580 7539 1252 1899 5139 1386 3870 4124 1...
199998,2,6405 3203 6644 983 794 1913 1678 5736 1397 191...,__label__2,6405 3203 6644 983 794 1913 1678 5736 1397 191...


In [58]:
test_df

,text,summary
0,5399 3117 1070 4321 4568 2621 5466 3772 4516 2...,5399 3117 1070 4321 4568 2621 5466 3772 4516 2...
1,2491 4109 1757 7539 648 3695 3038 4490 23 7019...,2491 4109 1757 7539 648 3695 3038 4490 23 7019...
2,2673 5076 6835 2835 5948 5677 3247 4124 2465 5...,2673 5076 6835 2835 5948 5677 3247 4124 2465 5...
3,4562 4893 2210 4761 3659 1324 2595 5949 4583 2...,4562 4893 2210 4761 3659 1324 2595 5949 4583 2...
4,4269 7134 2614 1724 4464 1324 3370 3370 2106 2...,4269 7134 2614 1724 4464 1324 3370 3370 2106 2...
...,...,...
49995,3725 4498 2282 1647 6293 4245 4498 3615 1141 2...,3725 4498 2282 1647 6293 4245 4498 3615 1141 2...
49996,4811 465 3800 1394 3038 2376 2327 5165 3070 57...,4811 465 3800 1394 3038 2376 2327 5165 3070 57...
49997,5338 1952 3117 4109 299 6656 6654 3792 6831 21...,5338 1952 3117 4109 299 6656 6654 3792 6831 21...
49998,893 3469 5775 584 2490 4223 6569 6663 2124 168...,893 3469 5775 584 2490 4223 6569 6663 2124 168...


In [61]:
#首尾截断进行训练
#train_df = pd.read_csv('./train_set.csv', sep='\t')
#train_df['label_ft'] = '__label__' + train_df['label'].astype(str)

train_df['summary']=train_df['text'].apply(lambda x:slipt2(x))

train_df[['summary','label_ft']].to_csv('train_summary_20w.csv', index=None, header=None, sep='\t')

model3 = fasttext.train_supervised('train_summary_20w.csv',pretrainedVectors='word_25w.vec',lr=0.8, wordNgrams=3, 
                                  verbose=2, minCount=1, epoch=18, loss="softmax")

val_pred = [model3.predict(x)[0][0].split('__')[-1] for x in train_df.iloc[-10000:]['summary']]

from sklearn.metrics import f1_score
print(f1_score(train_df['label'].values[-10000:].astype(str), val_pred, average='macro'))

#预测结果
test_df['summary']=test_df['text'].apply(lambda x:slipt2(x))
test_pred = [model3.predict(x)[0][0].split('__')[-1] for x in test_df['summary']]

pd.DataFrame({'label':test_pred}).to_csv('word_fast_cut_best.csv',index=None)

In [20]:
#看看不截断的f1分数
"""train_df[['text','label_ft']].iloc[:-10000].to_csv('train_test_19w.csv', index=None, header=None, sep='\t')

model3 = fasttext.train_supervised('train_test_19w.csv',pretrainedVectors='word_25w.vec',lr=0.8, wordNgrams=2, 
                                  verbose=2, minCount=1, epoch=18, loss="hs")"""
val_pred = [model3.predict(x)[0][0].split('__')[-1] for x in train_df.iloc[-10000:]['text']]

from sklearn.metrics import f1_score
print(f1_score(train_df['label'].values[-10000:].astype(str), val_pred, average='macro'))

0.9157807172377387


In [1]:
#正常fasttext应用以上参数：
import pandas as pd
from sklearn.metrics import f1_score

# 转换为FastText需要的格式
train_df = pd.read_csv('./train_set.csv', sep='\t')
train_df['label_ft'] = '__label__' + train_df['label'].astype(str)
train_df[['text','label_ft']].to_csv('train_20w.csv', index=None, header=None, sep='\t')

import fasttext
model = fasttext.train_supervised('train_20w.csv', lr=0.8, wordNgrams=3, 
                                  verbose=2, minCount=1, epoch=25, loss="softmax")


In [2]:
test_df = pd.read_csv('./test_a.csv', sep='\t')
test_pred = [model.predict(x)[0][0].split('__')[-1] for x in test_df['text']]

pd.DataFrame({'label':test_pred}).to_csv('fast2.csv',index=None)